In [ ]:
#!pip install tensorflow-gpu==1.15.0
#!pip install keras==2.2.5
#!pip install h5py==2.10.0 --force-reinstall
!pip install mrcnn
!pip install flask-ngrok

!git clone https://github.com/matterport/Mask_RCNN.git
%cd Mask_RCNN/
!python setup.py install
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

%cd ..
!git clone https://github.com/suso86/Deteccion-Automatica-Pinturas.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.15.0 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.12.0)
ERROR: No matching distribution found for tensorflow-gpu==1.15.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.2/336.2 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 KB 3.7 MB/s eta 0:00:00
  Attempting uninstall: keras

In [ ]:
%cd /content/

/content


In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.3 MB 7.5 MB/s 
  ERROR: Failed building wheel for numpy
  Running setup.py clean for numpy
Failed to build numpy
    Running setup.py install for numpy ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-g_e6g0pp/numpy_e6eeedadfd044cb4bc839b50321551ae/setup.py'"'"'; __file__='"'"'/tmp/pip-install-g_e6g0pp/numpy_e6eeedadfd044cb4bc839b50321551ae/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-kwwtopqe/install-record.txt --single-version-externally-managed --compile --install-headers /usr/loc

In [ ]:
import numpy
print(numpy.__version__)

1.21.6


In [ ]:
from flask.helpers import send_from_directory
from flask import Flask, render_template, request, redirect, url_for, make_response, jsonify
from werkzeug.utils import secure_filename
from os import remove
import os
import cv2
from flask_ngrok import run_with_ngrok
from datetime import timedelta
import damage_detection_colab
from keras import backend as K

!pip install pyngrok==4.1.1
!ngrok authtoken "2GZuKvg2gVgoOnoWyyTog01UDEY_54RgkWDSxGMhcGEVf4Vpp"


K.clear_session()

# Configuration of allowed file extensions.
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'JPG', 'PNG'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS

app = Flask(__name__)
run_with_ngrok(app)

# Set the expiration time for static files cache.
app.send_file_max_age_default = timedelta(seconds=1)

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/home")
def upload_image():
    return render_template("image_form.html")

@app.route("/home/artworks", methods=["POST"])
def detection():
    if request.method == "POST":

        f = request.files['file']
        name_file = f.filename

        if not (f and allowed_file(f.filename)):
            return jsonify({"error": 1001, "msg": "Please check the uploaded image type, only png, jpg, jpeg, JPG, PNG are allowed"})

        # name_file = request.form.get["name"]
        # Current file path
        basepath = '/content'

        images_folder = basepath + '/static/images/'
        upload_path = os.path.join(images_folder, f.filename)

        f.save(upload_path)

        img = cv2.imread(upload_path)

        # remove(upload_path)

        current_upload = os.path.join(images_folder, 'test.jpg')
        cv2.imwrite(current_upload, img)

        return render_template("damage_detection.html", name_file=name_file)

@app.route('/home/artworks/damage')
def show_damage():
    K.clear_session()
    name = request.args.get('name')

    basepath = '/content'

    # Since we are in Windows, we need to reverse the characters "/"
    images_folder = basepath + '/static/images/'
    box_path = images_folder + name

    # Once we have the image, we detect the damages
    zones = damage_detection_colab.saveImage_GetFeatures(name)

    image_size = zones[0]
    undamaged_zone = zones[1]
    damaged_zone = zones[2]
    percentage = zones[3]

    remove(box_path)

    return render_template("show_damage.html", name_file=name, size=image_size, undamaged_zone=undamaged_zone,
                           damaged_zone=damaged_zone, percentage=percentage)

@app.route('/home/artworks/damage/download')
def download_images():
    pathfile = '/content/static/images/'
    path = 'detection.jpg'
    return send_from_directory(pathfile, path, as_attachment=True)


app.run()
